In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from IPython.display import Audio
import librosa
import librosa.display
from tensorflow.contrib import rnn 

# TESTING RNN (TUTORIAL)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

# prints out full length of array
# np.set_printoptions(threshold=np.nan)

tfrecords_filename = 'D:/Documents/nsynth/nsynth-train.tfrecord'
filter_tfrecords_filename = 'D:/Documents/nsynth/filtered_data_keyboard.tfrecord'

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
# parse TFRecord file into a generator
# https://github.com/faroit/nsynth-convert
def nsynth_generator(tfrecords_filename):
    for serialized_example in tf.python_io.tf_record_iterator(tfrecords_filename):
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        f = example.features.feature
#         result = tf.train.Example.FromString(serialized_example)
#         print(result)

        audio = np.array(f['audio'].float_list.value)

        data = {
            'note':
                f['note'].int64_list.value[0],
            'note_str':
                f['note_str'].bytes_list.value[0],
            'instrument':
                f['instrument'].int64_list.value[0],
            'instrument_str':
                f['instrument_str'].bytes_list.value[0],
            'pitch':
                f['pitch'].int64_list.value[0],
            'velocity':
                f['pitch'].int64_list.value[0],
            'sample_rate':
                f['sample_rate'].int64_list.value[0],
            'audio':
                np.array(f['audio'].float_list.value),
            'qualities':
                np.array(f['qualities'].int64_list.value),
            'qualities_str':
                np.array(f['qualities_str'].bytes_list.value),
            'instrument_family':
                f['instrument_family'].int64_list.value[0],
            'instrument_family_str':
                f['instrument_family_str'].bytes_list.value[0],
            'instrument_source':
                f['instrument_family'].int64_list.value[0],
            'instrument_source_str':
                f['instrument_source_str'].bytes_list.value[0],
        }
#         print(data)

        yield data, audio

In [ ]:
# Filter function to only contain keyboard tag
def filterData(inputFile, outputFile):
    dataset = nsynth_generator(inputFile)
    count = 0
    data = next(dataset)
    writer = tf.python_io.TFRecordWriter(outputFile)
    try:
        while data:
            if data[1]["instrument_family"].int64_list.value[0] == 4:
                writer.write(data[0])
                print(count)
                print(data[2])
                count+=1
            data = next(dataset)
    except StopIteration:
        pass
    writer.close()

In [ ]:
# Testing nsynth_generator function
gen_samples = nsynth_generator(tfrecords_filename)
metadata, audio = next(gen_samples)

In [ ]:
# Filters the 80GB of data to new file total: 51820
outputFile = "filtered_data_keyboard.tfrecord"
filterData(tfrecords_filename, outputFile)

In [ ]:
# Print out all data
# for example in tf.python_io.tf_record_iterator(tfrecords_filename):
for example in tf.python_io.tf_record_iterator(filter_tfrecords_filename):
    result = tf.train.Example.FromString(example)
    print(result)

In [ ]:
gen_filter = nsynth_generator(filter_tfrecords_filename)

In [ ]:
# Filtered data
metadata, audio = next(gen_filter)
print(audio)
Audio(audio, rate=metadata['sample_rate'])
plt.plot(audio)

In [ ]:
X = librosa.cqt(audio)
Xdb = librosa.amplitude_to_db(abs(X))
# plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=22050, x_axis='time', y_axis='hz')

In [2]:
# RNN code
# hm_epochs = 10
# n_classes = 10
# batch_size = 20

# chunk_size = 250
# n_chunks = 256
# rnn_size = 128
hm_epochs = 3
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128

In [3]:
x = tf.placeholder('float', [None, n_chunks, chunk_size])
y = tf.placeholder('float')

In [4]:
def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)
#     print(x)

    lstm_cell = rnn.BasicLSTMCell(rnn_size,state_is_tuple=True) 
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output

In [5]:
def train_neural_network(x):
#     tf.reset_default_graph()
    prediction = recurrent_neural_network(x)
    print(prediction)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y) )
    print(cost)
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    print(optimizer)
    

    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))

train_neural_network(x)

Tensor("add:0", shape=(?, 10), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
name: "Adam"
op: "NoOp"
input: "^Adam/update_Variable/ApplyAdam"
input: "^Adam/update_Variable_1/ApplyAdam"
input: "^Adam/update_rnn/basic_lstm_cell/kernel/ApplyAdam"
input: "^Adam/update_rnn/basic_lstm_cell/bias/ApplyAdam"
input: "^Adam/Assign"
input: "^Adam/Assign_1"

Epoch 0 completed out of 3 loss: 205.22172967344522
Epoch 1 completed out of 3 loss: 57.23569322191179
Epoch 2 completed out of 3 loss: 39.01983759365976
Accuracy: 0.9742
